## Computing ERT geometric factors for a laboratory scale cylinder

In [23]:
# we first import pygimli and two modules, for meshing and for ERT
import pygimli as pg
import pygimli.meshtools as mt
from pygimli.physics import ert
import numpy as np

In [2]:
# define the electrodes and their spacings
nel = 8
de = 0.15
# define the dimensions of the cylinder
diameter = 0.0508
xLeft = 0.1
xRight = 0.0692
length = (nel-1) * de  + xLeft + xRight
print(length)

1.2192


In [91]:
shm = pg.DataContainerERT()
for n in range(nel):
    shm.createSensor([n*de, 0, -0.01])

for a in np.arange(nel-3):
    shm.createFourPointData(a, a, a+3, a+1, a+2)

print(shm)
print(pg.x(shm))

Data: Sensors: 8 data: 5, nonzero entries: ['a', 'b', 'm', 'n', 'valid']
8 [0.0, 0.15, 0.3, 0.44999999999999996, 0.6, 0.75, 0.8999999999999999, 1.05]


In [92]:
# We create a cylinder and shift it so that the 
from math import pi
radius = diameter/2
cyl = mt.createCylinder(radius=radius, height=length, pos=[0, -radius, length/2], nSegments=16)
print(cyl.boundingBox())
# cyl.rotate([0, pi/2, 0])
cyl.swapCoordinates(1, 2)
cyl.swapCoordinates(0, 1)
print(cyl.boundingBox())
cyl.translate([-xLeft, 0, 0])  # so that first electrode is at 0,0,0
print(cyl.boundingBox())
pg.show(cyl)

BoundingBox [RVector3: (-0.0254, -0.0508, 0.0), RVector3: (0.0254, 0.0, 1.2192)]
BoundingBox [RVector3: (0.0, -0.0254, -0.0508), RVector3: (1.2192, 0.0254, 0.0)]
BoundingBox [RVector3: (-0.1, -0.0254, -0.0508), RVector3: (1.1192, 0.0254, 0.0)]


(<pyvista.plotting.plotting.Plotter at 0x29f5e048c70>, None)

In [93]:
for el in shm.sensors():
    print(el)
    cyl.createNode(el, marker=-99)

RVector3: (0.0, 0.0, -0.01)
RVector3: (0.15, 0.0, -0.01)
RVector3: (0.3, 0.0, -0.01)
RVector3: (0.44999999999999996, 0.0, -0.01)
RVector3: (0.6, 0.0, -0.01)
RVector3: (0.75, 0.0, -0.01)
RVector3: (0.8999999999999999, 0.0, -0.01)
RVector3: (1.05, 0.0, -0.01)


In [94]:
print(cyl)
cyl.exportVTK("cylinder.vtk")
mesh = mt.createMesh(cyl, verbose=True)
print(mesh)
mesh.exportVTK("mesh.vtk")
#pg.show(mesh)

Mesh: Nodes: 40 Cells: 0 Boundaries: 18
tetgen -pzACaq1.2 C:\Users\Guenther.T\AppData\Local\Temp\tmp2hf1uij0.poly
Mesh: Nodes: 1857 Cells: 6910 Boundaries: 15173


In [97]:
meshP = mesh.createP2()
print(meshP)

Mesh: Nodes: 11976 Cells: 6910 Boundaries: 15173


In [98]:
# data = ert.simulate(mesh=meshP, scheme=shm)
k = ert.createGeometricFactors(shm, numerical=True, mesh=mesh, p2=True)
print(k)

04/10/22 - 12:36:06 - pyGIMLi - INFO - Save RVector binary
04/10/22 - 12:36:06 - pyGIMLi - INFO - Cache stored: C:\Guenther.T\Anwendungsdaten\pygimli\Cache\2841538644319391280


5 [0.013167537567750296, 0.013167537517369836, 0.013167537544046223, 0.013167537534940365, 0.01316753753298258]


In [100]:
kAna = radius**2*pi/de
print(kAna)
print(k/kAna*100-100)

0.013512199442599938
5 [-2.550745911602121, -2.55074628445378, -2.5507460870293244, -2.5507461544192296, -2.5507461689082334]
